In [20]:
#import pandas as pd
import pandas as pd



In [21]:
#Read in Data
Df = pd.read_csv('Resources/GoodReads_100k_books.csv')

In [22]:
#Display the Dataframe
Df.head()


,author,bookformat,desc,genre,img,isbn,isbn13,link,pages,rating,reviews,title,totalratings
0,Laurence M. Hauptman,Hardcover,Reveals that several hundred thousand Indians ...,"History,Military History,Civil War,American Hi...",https://i.gr-assets.com/images/S/compressed.ph...,002914180X,9.78E+12,https://goodreads.com/book/show/1001053.Betwee...,0,3.52,5,Between Two Fires: American Indians in the Civ...,33
1,"Charlotte Fiell,Emmanuelle Dirix",Paperback,Fashion Sourcebook - 1920s is the first book i...,"Couture,Fashion,Historical,Art,Nonfiction",https://i.gr-assets.com/images/S/compressed.ph...,1906863482,9.78E+12,https://goodreads.com/book/show/10010552-fashi...,576,4.51,6,Fashion Sourcebook 1920s,41
2,Andy Anderson,Paperback,The seminal history and analysis of the Hungar...,"Politics,History",https://i.gr-assets.com/images/S/compressed.ph...,948984147,9.78E+12,https://goodreads.com/book/show/1001077.Hungar...,124,4.15,2,Hungary 56,26
3,Carlotta R. Anderson,Hardcover,"""All-American Anarchist"" chronicles the life a...","Labor,History",https://i.gr-assets.com/images/S/compressed.ph...,814327079,9.78E+12,https://goodreads.com/book/show/1001079.All_Am...,324,3.83,1,All-American Anarchist: Joseph A. Labadie and ...,6
4,Jean Leveille,NaN,"Aujourdâ€™hui, lâ€™oiseau nous invite Ã sa ta...",NaN,https://i.gr-assets.com/images/S/compressed.ph...,2761920813,NaN,https://goodreads.com/book/show/10010880-les-o...,177,4.00,1,Les oiseaux gourmands,1


In [23]:
#Describe the Data
Df.columns.tolist() 

['author',
 'bookformat',
 'desc',
 'genre',
 'img',
 'isbn',
 'isbn13',
 'link',
 'pages',
 'rating',
 'reviews',
 'title',
 'totalratings']

In [24]:
#Drop unnecessary columns

Df = Df.drop(columns=['link','bookformat','img','isbn', 'isbn13'])

In [25]:
#Check if there are any null values

null_check = Df.isnull().sum()
print("Count of null values in each column:")
print(null_check)

Count of null values in each column:
author              0
desc             6772
genre           10467
pages               0
rating              0
reviews             0
title               1
totalratings        0
dtype: int64


In [26]:
#Drop rows with null values in target columns (desc and genre) 
Df = Df.dropna(subset=['desc', 'genre'])

In [27]:
genre_set=[]

for genres in Df["genre"]:
    genre_list = genres.split(",")
    for single_genre in genre_list:
        genre_set.append(single_genre)

genre_set = set(genre_set)

len(genre_set)


1179

In [28]:
Df["genre"] = Df["genre"].str.split(",")

Df.head()

,author,desc,genre,pages,rating,reviews,title,totalratings
0,Laurence M. Hauptman,Reveals that several hundred thousand Indians ...,"[History, Military History, Civil War, America...",0,3.52,5,Between Two Fires: American Indians in the Civ...,33
1,"Charlotte Fiell,Emmanuelle Dirix",Fashion Sourcebook - 1920s is the first book i...,"[Couture, Fashion, Historical, Art, Nonfiction]",576,4.51,6,Fashion Sourcebook 1920s,41
2,Andy Anderson,The seminal history and analysis of the Hungar...,"[Politics, History]",124,4.15,2,Hungary 56,26
3,Carlotta R. Anderson,"""All-American Anarchist"" chronicles the life a...","[Labor, History]",324,3.83,1,All-American Anarchist: Joseph A. Labadie and ...,6
5,Jeffrey Pfeffer,Why is common sense so uncommon when it comes ...,"[Business, Leadership, Romance, Historical Rom...",368,3.73,7,The Human Equation: Building Profits by Puttin...,119


In [29]:
#  Uncomment the line below if you are using Google Colab.
!pip install -U sentence-transformers

In [30]:
# Import the SentenceTransformer class and utility function class from the sentence_transformers module 
from sentence_transformers import SentenceTransformer, util
# Use the `all-MiniLM-L6-v2` model.
model = SentenceTransformer('all-MiniLM-L6-v2')
# Import pandas
import pandas as pd
pd.set_option('max_colwidth', 200)

In [ ]:
# Convert the "Desc" column to a list 
goodreads_desc  = Df["desc"].tolist()
goodreads_desc

In [33]:
# Get the vector embeddings for the descriptions.
goodreads_desc_embeddings = model.encode(goodreads_desc)

In [35]:
new_description = "Coming of age story in 19th century."
new_description_embedding = model.encode([new_description])

In [37]:
similarities = []
# Loop through the description embeddings.
for i, description_embedding in enumerate(new_description_embedding):
    # Calculate the cosine similarity score between each description embedding and the ew description embedding. 
    cosine_similarity_score = util.cos_sim(description_embedding, new_description_embedding)

    # Store the new description and similarity score as a tuple in the list.
    similarities.append((new_description[i], cosine_similarity_score))

# Sort the list of tuples based on similarity score in descending order
similarities.sort(key=lambda x: x[1], reverse=True)

In [46]:
# Print the sorted results
print(f"New description to categorize: {new_description}")
print()

# Loop through the similarities list and get the headline and similarity score.
for i, (description, similarity_score) in enumerate(similarities):
    # Get the genre from the DataFrame for each headline.
    genre = Df.loc[description == Df["desc"], "genre"].values[0][0]
    # Print the rank, genre, and the news headline.
    print(f"Rank {i+1}: Genre: {genre}, Description: {desc}")
    # Print the similarity score of the news headline.
    print(f"Similarity score: {similarity_score[0][0]}")
    print()

New description to categorize: Coming of age story in 19th century.



IndexError: index 0 is out of bounds for axis 0 with size 0